<img src='http://hilpisch.com/taim_logo.png' width="350px" align="right">

# Reinforcement Learning

## OpenAI Gym

Dr Yves J Hilpisch | The AI Machine

http://aimachine.io | http://twitter.com/dyjh

## CartPole

In [ ]:
!git clone https://github.com/tpq-classes/rl_for_finance.git
import sys
sys.path.append('rl_for_finance')


In [ ]:
import warnings
warnings.simplefilter('ignore')

In [ ]:
import gymnasium as gym
import numpy as np
import pandas as pd
from pylab import plt
from IPython import display
plt.style.use('seaborn-v0_8')
np.random.seed(100)

In [ ]:
env = gym.make('CartPole-v1')

In [ ]:
# env.seed(100)

## DNN Classification Approach

### Collecting Data

In [ ]:
data = pd.DataFrame()
state = env.reset()
length = []

In [ ]:
%%time
for run in range(25000):
    done = False
    prev_state, info = env.reset()
    treward = 1
    results = []
    while not done:
        action = env.action_space.sample()
        state, reward, done, trunc, _ = env.step(action)
        results.append({'s1': prev_state[0], 's2': prev_state[1],
                        's3': prev_state[2], 's4': prev_state[3],
                        'a': action, 'r': reward})
        treward += reward if not done else 0
        prev_state = state
    if treward >= 110:
        data = pd.concat((data, pd.DataFrame(results)))
        length.append(treward)

In [ ]:
np.array(length).mean()

In [ ]:
data.info()

In [ ]:
data.tail()

### Training the DNN

In [ ]:
#import tensorflow.compat.v1 as tf
import tensorflow as tf
from tensorflow.keras.layers import Dense
from tensorflow.keras.models import Sequential
#tf.random.set_random_seed(100)
#tf.logging.set_verbosity(tf.logging.ERROR)

In [ ]:
model = Sequential()
model.add(Dense(4 * 32, activation='relu',
                input_dim=env.observation_space.shape[0]))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['acc'])

In [ ]:
%%time
model.fit(data[['s1', 's2', 's3', 's4']], data['a'],
          epochs=25, verbose=False, validation_split=0.2)

In [ ]:
res = pd.DataFrame(model.history.history)
res.tail(3)

In [ ]:
res.plot(figsize=(10, 6), style='--');

### Testing the DNN Agent

In [ ]:
def epoch():
    done = False
    state, info = env.reset()
    treward = 1
    while not done:
        action = np.where(
            model(tf.convert_to_tensor(np.atleast_2d(state), dtype=tf.float32), training=False).numpy()[0][0] > 0.5, 1, 0)
        state, reward, done, trunc, _ = env.step(action)
        treward += reward if not done else 0
    return treward

In [ ]:
res = np.array([epoch() for _ in range(10)])
res

In [ ]:
res.mean()

<img src='http://hilpisch.com/taim_logo.png' width="350px" align="right">

<br><br><br><a href="http://tpq.io" target="_blank">http://tpq.io</a> | <a href="http://twitter.com/dyjh" target="_blank">@dyjh</a> | <a href="mailto:ai@tpq.io">ai@tpq.io</a>